In [3]:
import sys
from graphviz import Digraph, Source
from pathlib import Path

sys.path.append(str(Path.home() / 'Code/Python/wynini'))
from wynini import config as wconfig
from wynini import wfst
from wynini.wfst import *

help(wconfig)

Help on module wynini.config in wynini:

NAME
    wynini.config - # -*- coding: utf-8 -*-

FUNCTIONS
    init(config)
        Set globals with dictionary or module

DATA
    bos = '⋊'
    eos = '⋉'
    epsilon = 'ϵ'
    sigma = ['a', 'b']
    special_syms = []
    syms = []
    symtable = None
    unk = '⊥'
    verbosity = 0
    λ = ''

FILE
    /Users/colin/Code/Python/wynini/wynini/config.py




In [15]:
#M = braid(2)
M = trellis(2)
#M = ngram(context='left', length=1)
M.draw('M.dot')
M_viz = Source.from_file('M.dot')
M_viz

NameError: name 'lattice' is not defined

In [14]:
M.draw()

TypeError: Wfst.draw() missing 1 required positional argument: 'source'